In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

In [ ]:
# Basic VAE 
class VAE(nn.Module):
    def __init__(self, input_dim=20, hidden_dim=64, latent_dim=10):
        super(VAE, self).__init__()

        # Encoder
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU()
        )
        self.fc_mu = nn.Linear(hidden_dim // 2, latent_dim)
        self.fc_logvar = nn.Linear(hidden_dim // 2, latent_dim)

        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Linear(hidden_dim // 2, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, input_dim),
            nn.Sigmoid()
        )

    def encode(self, x):
        h = self.encoder(x)
        mu = self.fc_mu(h)
        logvar = self.fc_logvar(h)
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        recon_x = self.decode(z)
        return recon_x, mu, logvar

In [6]:
def loss_function(recon_x, x, mu, logvar):
    bce = nn.BCELoss(reduction='sum')(recon_x, x)
    kld = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return bce + kld

In [ ]:
def generate_palindromic_binary_data(num_samples=1000, length=20):
    data = []
    for _ in range(num_samples):
        half = np.random.randint(0, 2, size=length // 2)
        palindrome = np.concatenate((half, half[::-1]))
        data.append(palindrome)
    return torch.tensor(data, dtype=torch.float32)
def is_palindrome(seq):
    return seq == seq[::-1]

In [ ]:
binary_palindromes = generate_palindromic_binary_data()
dataloader = DataLoader(TensorDataset(binary_palindromes), batch_size=64, shuffle=True)

In [ ]:
vae = VAE()
optimizer = optim.Adam(vae.parameters(), lr=0.001)

In [ ]:
def train(epochs=100):
    vae.train()
    for epoch in range(epochs):
        total_loss = 0
        for batch in dataloader:
            x = batch[0]
            optimizer.zero_grad()
            recon_x, mu, logvar = vae(x)
            loss = loss_function(recon_x, x, mu, logvar)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {total_loss / len(dataloader.dataset):.4f}")

Epoch 1, Loss: 13.9530
Epoch 2, Loss: 13.8792
Epoch 3, Loss: 13.8661
Epoch 4, Loss: 13.8645
Epoch 5, Loss: 13.8622
Epoch 6, Loss: 13.8670
Epoch 7, Loss: 13.8561
Epoch 8, Loss: 13.8602
Epoch 9, Loss: 13.8677
Epoch 10, Loss: 13.8582
Epoch 11, Loss: 13.8626
Epoch 12, Loss: 13.8572
Epoch 13, Loss: 13.8595
Epoch 14, Loss: 13.8573
Epoch 15, Loss: 13.8539
Epoch 16, Loss: 13.8624
Epoch 17, Loss: 13.8426
Epoch 18, Loss: 13.8567
Epoch 19, Loss: 13.8437
Epoch 20, Loss: 13.8373
Epoch 21, Loss: 13.8361
Epoch 22, Loss: 13.8158
Epoch 23, Loss: 13.8227
Epoch 24, Loss: 13.8188
Epoch 25, Loss: 13.7771
Epoch 26, Loss: 13.7810
Epoch 27, Loss: 13.7657
Epoch 28, Loss: 13.7144
Epoch 29, Loss: 13.7247
Epoch 30, Loss: 13.7023
Epoch 31, Loss: 13.6464
Epoch 32, Loss: 13.5765
Epoch 33, Loss: 13.6254
Epoch 34, Loss: 13.6524
Epoch 35, Loss: 13.6120
Epoch 36, Loss: 13.5025
Epoch 37, Loss: 13.5364
Epoch 38, Loss: 13.4724
Epoch 39, Loss: 13.5550
Epoch 40, Loss: 13.4663
Epoch 41, Loss: 13.4867
Epoch 42, Loss: 13.5668
E

In [ ]:
train(epochs=100)

Epoch 1, Loss: 11.5439
Epoch 2, Loss: 11.5542
Epoch 3, Loss: 11.5475
Epoch 4, Loss: 11.6176
Epoch 5, Loss: 11.6832
Epoch 6, Loss: 11.4448
Epoch 7, Loss: 11.4635
Epoch 8, Loss: 11.6732
Epoch 9, Loss: 11.6002
Epoch 10, Loss: 11.5491
Epoch 11, Loss: 11.5418
Epoch 12, Loss: 11.4843
Epoch 13, Loss: 11.5603
Epoch 14, Loss: 11.5840
Epoch 15, Loss: 11.5080
Epoch 16, Loss: 11.5318
Epoch 17, Loss: 11.5162
Epoch 18, Loss: 11.5105
Epoch 19, Loss: 11.6413
Epoch 20, Loss: 11.5528
Epoch 21, Loss: 11.5645
Epoch 22, Loss: 11.6402
Epoch 23, Loss: 11.6034
Epoch 24, Loss: 11.4565
Epoch 25, Loss: 11.5290
Epoch 26, Loss: 11.5416
Epoch 27, Loss: 11.5858
Epoch 28, Loss: 11.5257
Epoch 29, Loss: 11.5250
Epoch 30, Loss: 11.4942
Epoch 31, Loss: 11.5171
Epoch 32, Loss: 11.5146
Epoch 33, Loss: 11.4891
Epoch 34, Loss: 11.6004
Epoch 35, Loss: 11.4472
Epoch 36, Loss: 11.5298
Epoch 37, Loss: 11.4153
Epoch 38, Loss: 11.6248
Epoch 39, Loss: 11.5264
Epoch 40, Loss: 11.4928
Epoch 41, Loss: 11.4722
Epoch 42, Loss: 11.4540
E

Epoch 339, Loss: 9.6978
Epoch 340, Loss: 9.5877
Epoch 341, Loss: 9.6999
Epoch 342, Loss: 9.7238
Epoch 343, Loss: 9.7009
Epoch 344, Loss: 9.6558
Epoch 345, Loss: 9.6969
Epoch 346, Loss: 9.5256
Epoch 347, Loss: 9.5845
Epoch 348, Loss: 9.4831
Epoch 349, Loss: 9.5867
Epoch 350, Loss: 9.4922
Epoch 351, Loss: 9.5940
Epoch 352, Loss: 9.5156
Epoch 353, Loss: 9.5144
Epoch 354, Loss: 9.5949
Epoch 355, Loss: 9.5940
Epoch 356, Loss: 9.5393
Epoch 357, Loss: 9.5097
Epoch 358, Loss: 9.4882
Epoch 359, Loss: 9.4992
Epoch 360, Loss: 9.6694
Epoch 361, Loss: 9.5666
Epoch 362, Loss: 9.7904
Epoch 363, Loss: 9.6879
Epoch 364, Loss: 9.4204
Epoch 365, Loss: 9.5461
Epoch 366, Loss: 9.3454
Epoch 367, Loss: 9.5903
Epoch 368, Loss: 9.6057
Epoch 369, Loss: 9.4439
Epoch 370, Loss: 9.5094
Epoch 371, Loss: 9.5462
Epoch 372, Loss: 9.5874
Epoch 373, Loss: 9.4993
Epoch 374, Loss: 9.5043
Epoch 375, Loss: 9.4373
Epoch 376, Loss: 9.6997
Epoch 377, Loss: 9.4403
Epoch 378, Loss: 9.4179
Epoch 379, Loss: 9.4765
Epoch 380, Loss:

In [ ]:
vae.eval()

with torch.no_grad():
    z = torch.randn(100, 10)  # 100 samples from the latent space
    generated = vae.decode(z).round()
    generated_sequences = [''.join(map(str, seq.int().tolist())) for seq in generated]

    correct_palindromes = [seq for seq in generated_sequences if is_palindrome(seq)]

    print("Generated binary palindromic sequences:")
    for seq in generated_sequences:
        print(seq)

    print(f"\nTotal generated: {len(generated_sequences)}")
    print(f"Correct palindromic sequences: {len(correct_palindromes)}")
    print(f"Accuracy: {len(correct_palindromes) / len(generated_sequences) * 100:.2f}%")

Generated binary palindromic sequences:
10001001011010010001
11000001100110000011
01011001100110011010
11110011000011001111
01010101100110101010
00111101011010111100
01101101111110110110
00101100100100110101
11011010100101011011
01001111100111110010
00010001100110001000
00101110111101110100
11000111011011100011
01100111011011100110
01011100111100011010
00010000100100001000
10100110000001100101
00011111000011111000
11001001100110010011
11101001011010010111
01000000000000000010
11100010000001000111
01100100100100100110
11101000011000010111
00010011000011001000
00111110011001111100
10011010100101011001
01101110011001110110
00101011100111010100
11010101000010101011
10001001000010110001
10101000100100010101
10101011011011010101
10001010100101010001
11101011011011010111
01010111000011101010
01000001100110000010
10000010100101000001
11010010111101001011
11100000111100000111
01111100000000111110
00101110011001010100
11001010111101010011
10100010000001000101
00000001100110000000
000010111111110

In [26]:
generated_sequences

100

In [ ]:
s